# ML Model Horse Racing Predictions
#### Create an ML model that can predict whether a horse has a good chance of winning a race, based on the horse's features

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
cleaned_2020_2019_df = pd.read_csv("Data_Racing/International_Data/cleaned_2020_2019_df.csv")
cleaned_2020_2019_df.head()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-07-19 19:41:41.674105: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Unnamed: 0,ages,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,father,weight,res_win
0,0,3yo+,1407.0,1,5.0,3.0,0.200000,0,63.0,NaN,56.0,Makfi,57,0.0
1,1,3yo+,1407.0,1,6.0,7.0,0.333333,1,67.0,NaN,53.0,Commands,56,0.0
2,2,3yo+,1407.0,1,3.0,11.0,0.004587,0,49.0,NaN,49.0,All Too Hard,54,0.0
3,5,3yo+,1407.0,1,4.0,9.0,0.027778,0,46.0,NaN,52.0,Showcasing,55,0.0
4,7,3yo+,1407.0,1,4.0,2.0,0.004975,0,45.0,NaN,57.0,Kodiac,57,0.0


In [2]:
cleaned_2020_2019_df = cleaned_2020_2019_df.drop(columns = "Unnamed: 0")

In [3]:
# Determine the number of unique values in each column.
cleaned_2020_2019_df.nunique()

ages             24
metric           61
ncond            18
age              15
saddle           39
decimalPrice    600
isFav             2
RPR             178
TR              170
OR              173
father          413
weight           36
res_win           2
dtype: int64

## Select equal res_win values

In [5]:
# Look at value counts for binning
res_win_counts = cleaned_2020_2019_df["res_win"].value_counts()
class_count = res_win_counts.min()
class_count

26108

In [19]:
res_win_equal = pd.concat([cleaned_2020_2019_df[cleaned_2020_2019_df.res_win==0].sample(n=class_count), 
                           cleaned_2020_2019_df[cleaned_2020_2019_df.res_win==1].sample(n=class_count)])
res_win_equal = res_win_equal.sort_index()
res_win_equal = res_win_equal.dropna()
res_win_equal

,ages,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,father,weight,res_win
10,5yo+,4827.0,10,9.0,1.0,0.090909,0,129.0,105.0,132.0,Robin Des Champs,76,0.0
12,5yo+,4827.0,10,10.0,7.0,0.058824,0,95.0,75.0,119.0,Alflora,70,0.0
15,4yo+,3921.5,1,8.0,4.0,0.421053,0,123.0,72.0,130.0,Indian River,69,1.0
20,4yo+,3116.5,1,5.0,7.0,0.111111,0,128.0,113.0,120.0,Shamardal,66,1.0
29,5yo+,5028.0,1,8.0,2.0,0.250000,0,131.0,86.0,120.0,Presenting,69,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253195,4yo+,3116.5,5,5.0,1.0,0.117647,0,121.0,100.0,127.0,Kapgarde,76,0.0
253196,4yo+,4022.0,5,7.0,5.0,0.153846,0,144.0,82.0,129.0,Poliglote,70,1.0
253197,4yo+,4022.0,5,7.0,4.0,0.117647,0,137.0,75.0,129.0,Midnight Legend,70,0.0
253205,4yo+,4725.5,9,8.0,1.0,0.076923,0,141.0,3.0,139.0,Shantou,75,1.0


## Convert and Standardize Data

In [20]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_data = pd.get_dummies(res_win_equal)
converted_data.head()

,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,weight,...,father_Whipper,father_Whitmore's Conn,father_Winged Love,father_Wootton Bassett,father_Yeats,father_Youmzain,father_Zambezi Sun,father_Zanzibari,father_Zebedee,father_Zoffany
10,4827.0,10,9.0,1.0,0.090909,0,129.0,105.0,132.0,76,...,0,0,0,0,0,0,0,0,0,0
12,4827.0,10,10.0,7.0,0.058824,0,95.0,75.0,119.0,70,...,0,0,0,0,0,0,0,0,0,0
15,3921.5,1,8.0,4.0,0.421053,0,123.0,72.0,130.0,69,...,0,0,0,0,0,0,0,0,0,0
20,3116.5,1,5.0,7.0,0.111111,0,128.0,113.0,120.0,66,...,0,0,0,0,0,0,0,0,0,0
29,5028.0,1,8.0,2.0,0.250000,0,131.0,86.0,120.0,69,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Split our preprocessed data into our features and target arrays
y = converted_data['res_win']
X = converted_data.drop(columns=['res_win'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
input_features = len(X_train_scaled[0])
input_features

406

## Compile, Train and Evaluate the Model

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
neurons_hidden_layer1 = 456
neurons_hidden_layer2 = 328
neurons_hidden_layer3 = 176

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer1, activation="relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 456)               185592    
                                                                 
 dense_5 (Dense)             (None, 328)               149896    
                                                                 
 dense_6 (Dense)             (None, 176)               57904     
                                                                 
 dense_7 (Dense)             (None, 1)                 177       
                                                                 
Total params: 393,569
Trainable params: 393,569
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 150)

Epoch 1/150
492/492 [==============================] - 1s 2ms/step - loss: 0.5746 - accuracy: 0.7006
Epoch 2/150
492/492 [==============================] - 1s 2ms/step - loss: 0.4732 - accuracy: 0.7732
Epoch 3/150
492/492 [==============================] - 1s 2ms/step - loss: 0.4130 - accuracy: 0.8099
Epoch 4/150
492/492 [==============================] - 1s 2ms/step - loss: 0.3734 - accuracy: 0.8279
Epoch 5/150
492/492 [==============================] - 1s 2ms/step - loss: 0.3499 - accuracy: 0.8421
Epoch 6/150
492/492 [==============================] - 1s 2ms/step - loss: 0.3313 - accuracy: 0.8509
Epoch 7/150
492/492 [==============================] - 1s 2ms/step - loss: 0.3143 - accuracy: 0.8609
Epoch 8/150
492/492 [==============================] - 1s 2ms/step - loss: 0.3027 - accuracy: 0.8646
Epoch 9/150
492/492 [==============================] - 1s 2ms/step - loss: 0.2916 - accuracy: 0.8715
Epoch 10/150
492/492 [==============================] - 1s 2ms/step - loss: 0.2821 - accura

492/492 [==============================] - 1s 2ms/step - loss: 0.0164 - accuracy: 0.9938
Epoch 82/150
492/492 [==============================] - 1s 2ms/step - loss: 0.0240 - accuracy: 0.9919
Epoch 83/150
492/492 [==============================] - 1s 2ms/step - loss: 0.0393 - accuracy: 0.9877
Epoch 84/150
492/492 [==============================] - 1s 2ms/step - loss: 0.0384 - accuracy: 0.9873
Epoch 85/150
492/492 [==============================] - 1s 2ms/step - loss: 0.0307 - accuracy: 0.9910
Epoch 86/150
492/492 [==============================] - 1s 2ms/step - loss: 0.0259 - accuracy: 0.9914
Epoch 87/150
492/492 [==============================] - 1s 2ms/step - loss: 0.0215 - accuracy: 0.9933
Epoch 88/150
492/492 [==============================] - 1s 2ms/step - loss: 0.0235 - accuracy: 0.9928
Epoch 89/150
492/492 [==============================] - 1s 2ms/step - loss: 0.0264 - accuracy: 0.9915
Epoch 90/150
492/492 [==============================] - 1s 2ms/step - loss: 0.0255 - accuracy: 

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

164/164 - 0s - loss: 1.4726 - accuracy: 0.8227 - 292ms/epoch - 2ms/step
Loss: 1.4726359844207764, Accuracy: 0.8226760625839233


In [33]:
y_test.mean()

0.5651841954571483

In [29]:
# Make predictions on the test data
y_pred = nn.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)

164/164 [==============================] - 0s 671us/step


In [30]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_binary)
print("Precision:", precision)

Precision: 0.8479452054794521


In [31]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

         0.0       0.79      0.81      0.80      2278
         1.0       0.85      0.84      0.84      2961

    accuracy                           0.82      5239
   macro avg       0.82      0.82      0.82      5239
weighted avg       0.82      0.82      0.82      5239



In [18]:
# Export our model to HDF5 file
# nn.save('h5_files/AlphabetSoup_Model4.h5')